In [3]:
!pip install plotly_express==0.4.0

In [4]:
# Zipcodes for all cities in Washington State
from washington_cities import locations

# Print variables
print(locations)

{'Aberdeen city': [46.98, -123.81], 'Aberdeen Gardens': [47.06, -123.78], 'Acme': [48.71, -122.23], 'Addy': [48.36, -117.84], 'Ahtanum': [46.56, -120.61], 'Airway Heights city': [47.64, -117.58], 'Albion town': [46.79, -117.25], 'Alder': [46.79, -122.27], 'Alderton': [47.17, -122.22], 'Alderwood Manor': [47.81, -122.27], 'Alger': [48.62, -122.34], 'Algona city': [47.28, -122.25], 'Allyn': [47.39, -122.84], 'Almira town': [47.71, -118.94], 'Altoona': [46.27, -123.61], 'Amanda Park': [47.45, -123.92], 'Amboy': [45.91, -122.48], 'Ames Lake': [47.63, -121.97], 'Anacortes city': [48.49, -122.62], 'Anderson Island': [47.16, -122.7], 'Arlington city': [48.17, -122.15], 'Arlington Heights': [48.21, -122.06], 'Artondale': [47.3, -122.64], 'Ashford': [46.75, -122.01], 'Asotin city': [46.34, -117.04], 'Auburn city': [47.3, -122.21], 'Bainbridge Island city': [47.65, -122.52], 'Bangor Base': [47.72, -122.71], 'Banks Lake South': [47.63, -119.28], 'Barberton': [45.71, -122.61], 'Baring': [47.77, -1

In [5]:
import pandas as pd
from google.colab import files

uploaded = files.upload()

import io

# Data file for all registered electric vehicles in the state of Washington
df = pd.read_csv(io.BytesIO(uploaded['Electric_Vehicle_Data.csv']))

# Print dataframe
print(df)

Saving Electric_Vehicle_Data.csv to Electric_Vehicle_Data.csv
        VIN (1-10)    County               City State  Postal Code  \
0       2C4RC1N71H    Kitsap          Bremerton    WA      98311.0   
1       2C4RC1N7XL   Stevens           Colville    WA      99114.0   
2       KNDC3DLCXN    Yakima             Yakima    WA      98908.0   
3       5YJ3E1EA0J    Kitsap  Bainbridge Island    WA      98110.0   
4       1N4AZ1CP7J  Thurston           Tumwater    WA      98501.0   
...            ...       ...                ...   ...          ...   
159462  KM8JBDA2XP  Skamania          Underwood    WA      98651.0   
159463  1G1FZ6S02M    Skagit                Bow    WA      98232.0   
159464  YV4H60CX2P      King          Sammamish    WA      98029.0   
159465  5YJ3E1EA7K   Whatcom         Bellingham    WA      98225.0   
159466  7SAYGDEF6N    Island      Camano Island    WA      98282.0   

        Model Year       Make     Model  \
0             2017   CHRYSLER  PACIFICA   
1          

In [10]:
import plotly_express as px
import plotly.graph_objects as go

def heatmap(dct):
    """
    Parameters:
    dct: Dictionary with cities as keys and their electric vehicle counts as values

    Create a heatmap using Plotly Express with city coordinates and electric vehicle counts

    """
    cities = []
    ev_pop = []
    lat = []
    long = []

    # Iterate through each city in the provided dictionary
    for city in dct:

        # Iterate through each city_lat_long in the 'locations' dictionary
        for city_lat_long in locations:

            # Check if the city_lat_long starts with the current city
            if city_lat_long.startswith(city):

                # Append latitude, longitude, city, and electric vehicle count to respective lists
                lat.append(locations[city_lat_long][0])
                long.append(locations[city_lat_long][1])
                cities.append(city)
                ev_pop.append(dct[city])

    # Create a DataFrame for Plotly Express
    data = {
        'City': cities,
        'EV_Population': ev_pop,
        'Latitude': lat,
        'Longitude': long
    }

    df = pd.DataFrame(data)

    # Create a density map using Plotly Express
    fig = px.density_mapbox(
        df,
        lat='Latitude',
        lon='Longitude',
        z='EV_Population',
        radius=40,
        center=dict(lat=df['Latitude'].mean(), lon=df['Longitude'].mean()),
        zoom=6,
        mapbox_style='open-street-map',
        height=700
    )

    # Add a scatter plot on top of the density map to represent each city's EV population with markers
    # Added to visually represent cities with smaller EV populations
    fig.add_trace(
        go.Scattermapbox(
            lat=df["Latitude"],
            lon=df["Longitude"],
            mode="markers",
            showlegend=False,
            hoverinfo="skip",
            marker={
                "color": df["EV_Population"],
                "size": df["EV_Population"].fillna(0),
                "coloraxis": "coloraxis",
                "sizeref": (df["EV_Population"].max()) / 15 ** 2,
                "sizemode": "area",
            },
        )
    )

    # Display the heatmap
    fig.show()

# Count the occurrences of each city in the DataFrame and create a heatmap
category_counts = df['City'].value_counts().to_dict()
heatmap(category_counts)